# Chains in LangChain

## Outline
* LLMChain
* Sequential Chains
* SimpleSequentialChain
* SequentialChain
* Router Chain

In [5]:
import warnings
warnings.filterwarnings('ignore')

import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

api_key = os.environ['GROQ']

In [2]:
import pandas as pd

df = pd.read_csv('Data.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Data.csv'

## LLMChain

In [3]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [6]:

llm = ChatGroq(model="llama3-8b-8192", temperature=0.9, api_key=api_key)

In [7]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)
chain = LLMChain(llm=llm, prompt=prompt)
product = "Queen Size Sheet Set"
chain.run(product)

"What a royal question! Here are some name suggestions for a company that makes Queen Size Sheet Set:\n\n1. **Regal Linens**: This name plays off the idea of queens having lavish linens, which fits perfectly with your product.\n2. **Queenly Dreams**: This name evokes a sense of comfort and coziness, suggesting that your sheet sets will help people sleep like royalty.\n3. **The Royal Treatment**: This name positions your company as offering a luxurious experience, implying that your sheet sets are the epitome of comfort and quality.\n4. **Sheets of Majesty**: Another playful name that references the regal nature of your product, while also highlighting its quality.\n5. **Luxurious Slumber**: This name focuses on the comfort and relaxation aspect of your sheet sets, implying that they will help people sleep luxuriously.\n6. **The Queen's Chamber**: This name has a regal, elegant sound and suggests that your sheet sets are fit for a queen (or king)!\n7. **Paragon Linens**: This name conve

## SimpleSequentialChain

In [8]:
from langchain.chains import SimpleSequentialChain
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [9]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [10]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [11]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
What a great question!

Here are some suggestions for a company name that specializes in queen-size sheet sets:

1. **Regal Bedding Co.**: "Regal" implies luxury and grandeur, perfect for a queen-size sheet set.
2. **Queen's Quarters Linens**: This name plays on the idea of the queen's quarters and the queen-size bed sheets.
3. **Royal Textiles**: "Royal" conveys a sense of quality and nobility, while "Textiles" highlights the focus on fabric.
4. **Canopy Comfort**: "Canopy" evokes a sense of grandeur, while "Comfort" emphasizes the cozy nature of the sheet sets.
5. **Luna Luxe Linens**: "Luna" means moon, which is often associated with the queen-size bed, while "Luxe" implies high-end quality.
6. **Her Majesty's Sheets**: A playful nod to the royal theme, with a touch of whimsy.
7. **Comfort Castle Bedding**: "Comfort" emphasizes the cozy nature of the sheet sets, while "Castle" suggests a regal, luxurious feel.
8. **Bed of Dreams**: Thi

'Here is a 20-word description for Regal Bedding Co.:\n\n"Regal Bedding Co. crafts luxurious queen-size sheet sets with precision, comfort, and elegance, elevating your sleep experience to royal standards every night."'

## SequentialChain

In [12]:
from langchain.chains import SequentialChain

In [13]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

In [14]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [15]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [16]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [17]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [19]:
review = df.Review[5]
overall_chain(review)

NameError: name 'df' is not defined

## Router Chain

In [20]:
physics_template = """You are a very smart physics professor. 
You are great at answering questions about physics in a concise
and easy to understand manner. 
When you don't know the answer to a question you admit
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. 
You are great at answering math questions. 
You are so good because you are able to break down 
hard problems into their component parts, 
answer the component parts, and then put them together
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. 
You have an excellent knowledge of and understanding of people,
events and contexts from a range of historical periods. 
You have the ability to think, reflect, debate, discuss and 
evaluate the past. You have a respect for historical evidence
and the ability to make use of it to support your explanations 
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.
You have a passion for creativity, collaboration,
forward-thinking, confidence, strong problem-solving capabilities,
understanding of theories and algorithms, and excellent communication 
skills. You are great at answering coding questions. 
You are so good because you know how to solve a problem by 
describing the solution in imperative steps 
that a machine can easily interpret and you know how to 
choose a solution that has a good balance between 
time complexity and space complexity. 

Here is a question:
{input}"""

In [21]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [23]:
llm = ChatGroq(model="llama3-8b-8192", temperature=0, api_key=api_key)

In [24]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [25]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [26]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a 
language model select the model prompt best suited for the input. 
You will be given the names of the available prompts and a 
description of what the prompt is best suited for. 
You may also revise the original input if you think that revising
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of 
the candidate prompts listed below.
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input 
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [27]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [28]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [29]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation in the context of thermodynamics?'}
> Finished chain.


"Black body radiation is a fundamental concept in thermodynamics, and it's a fascinating topic.\n\nIn simple terms, black body radiation refers to the thermal radiation emitted by an idealized object, known as a blackbody, when it's heated to a high temperature. The key characteristic of a blackbody is that it absorbs all incident electromagnetic radiation, regardless of its frequency or polarization.\n\nWhen a blackbody is heated, it begins to emit radiation across a wide range of frequencies, from low-frequency radio waves to high-frequency gamma rays. This radiation is known as blackbody radiation. The emitted radiation is characterized by a specific distribution of energy, known as the Planck distribution, which was derived by Max Planck in the early 20th century.\n\nThe Planck distribution shows that the energy emitted by a blackbody is directly proportional to the fourth power of its temperature (T). This means that as the temperature of the blackbody increases, the amount of rad

In [30]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'A classic!\n\nTo answer this question, I\'ll break it down into its component parts:\n\n1. What is the definition of addition?\n\nAddition is a binary operation that combines two numbers to produce a sum. In this case, we\'re adding 2 and 2.\n\n2. What are the values of the numbers being added?\n\nThe values are: 2 and 2.\n\n3. How do we combine these values using the definition of addition?\n\nTo combine these values, we simply count up the total number of units:\n\n2 + 2 = ?\n\nWe start with 2 units, and then add 2 more units. This gives us a total of:\n\n2 + 2 = 4\n\nSo, the answer to the question "what is 2 + 2" is:\n\n2 + 2 = 4'

In [31]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'